In [1]:
import sys
sys.path.append('/Users/victoria/Documents/Scripts/Python/DGL-PTM/dgl_ptm')
import torch
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


This version features the desired income generation function; a fitted function is used as a decent placeholder to determine consumption. There are many aspects (such as optimization and personal perception of theta) which will be required in future but are currently not utilized. With the model in its current arrangement, the agents stop trading and stabillize after ~7 time steps. I have included a commented-out section at the bottom which uses matplotlib pyplot if you wish to get a quick look.
The current version of the config.yaml file is not formatted correctly and should not be used as is.

In [2]:
model = dgl_ptm.PovertyTrapModel(model_identifier='testwithnn')

In [3]:
model.set_model_parameters(default=False, **{'npath':'./agent_data_s.zarr','number_agents': 10000,'step_target': 10, 'steering_parameters': {'npath':'./agent_data.zarr',
                            'epath':'./edge_data', 
                            'ndata':['all_except',['a_table']],
                            'edata':['all'],
                            'mode':'xarray',
                            'wealth_method':'singular_transfer',
                            'income_method':'income_generation',
                            'tech_gamma': torch.tensor([0.3,0.35,0.45]),
                            'tech_cost': torch.tensor([0,0.15,0.65]),
                            'consume_method':'estimated_bellman_consumption',
                            'nn_path':'/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/nn_data/both_PudgeFiveLayer_1024/0409_175055/model_best.pth',
                            'adapt_m':torch.tensor([0,0.5,0.9]),
                            'adapt_cost':torch.tensor([0,0.25,0.45]),
                            'depreciation': 0.6,
                            'discount': 0.95,
                            'm_theta_dist': {'type':'multinomial','parameters':[[0.01 ,0.1, 0.79, 0.1],[0.1, 0.5, 0.8, 1]],'round':False,'decimals':None},
                            'del_prob':0.05,
                            'ratio':0.1,
                            'weight_a':0.69,
                            'weight_b':35, 
                            'truncation_weight':1.0e-10,
                            'step_type': 'custom'}})

#print(model.number_agents)  
#print(model.nn_path)  
print(model.steering_parameters)                        

{'npath': './agent_data.zarr', 'epath': './edge_data', 'ndata': ['all_except', ['a_table']], 'edata': ['all'], 'mode': 'xarray', 'wealth_method': 'singular_transfer', 'income_method': 'income_generation', 'tech_gamma': tensor([0.3000, 0.3500, 0.4500]), 'tech_cost': tensor([0.0000, 0.1500, 0.6500]), 'consume_method': 'estimated_bellman_consumption', 'nn_path': '/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/nn_data/both_PudgeFiveLayer_1024/0409_175055/model_best.pth', 'adapt_m': tensor([0.0000, 0.5000, 0.9000]), 'adapt_cost': tensor([0.0000, 0.2500, 0.4500]), 'depreciation': 0.6, 'discount': 0.95, 'm_theta_dist': {'type': 'multinomial', 'parameters': [[0.01, 0.1, 0.79, 0.1], [0.1, 0.5, 0.8, 1]], 'round': False, 'decimals': None}, 'del_prob': 0.05, 'ratio': 0.1, 'weight_a': 0.69, 'weight_b': 35, 'truncation_weight': 1e-10, 'step_type': 'custom'}


In [4]:
model.initialize_model()

In [5]:
model.run()



performing step 1 of 10
Initial k
tensor([9.0907, 6.9009, 4.6598,  ..., 9.6247, 0.1081, 2.8128])
dst node 9517 is an end node
no FoF link possible for src dst nodes 5725 and 9517.
created 0 of 1 links requested
entered load_consumption_model
loaded info
loaded state dict
 went forward, writing values
Cleaning output and checking for violations
Agents in violation: 1033
...because of i_a: 0
gets to data collection
performing step 2 of 10
Initial k
tensor([9.0907, 6.9009, 4.6598,  ..., 9.6247, 0.1081, 2.8128])
creating bidirectional link between nodes 7390 (src) and tensor([22]) (dst)
created 1 of 1 links requested
entered load_consumption_model
loaded info
loaded state dict
 went forward, writing values
Cleaning output and checking for violations
Agents in violation: 107
...because of i_a: 0
gets to data collection
performing step 3 of 10
Initial k
tensor([4.7028, 3.9098, 1.9354,  ..., 0.0487, 0.1699, 2.0149])
dst node 4736 is an end node
no FoF link possible for src dst nodes 1329 and 

In [7]:
'''
# try to see something happening
import zarr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation



agent_data = zarr.open('/Users/victoria/Documents/Scripts/Python/DGL-PTM/DGL_testing/agent_data.zarr', mode='r')
alpha=pd.DataFrame(agent_data['alpha'])
net_trade=pd.DataFrame(agent_data['net_trade'])
k=pd.DataFrame(agent_data['wealth'])

fig=plt.figure()
plt.xlabel("α")
plt.ylabel("$k_{t}$")
#xlims=[0.8,OGdata.loc[TimeSample,"alpha"].max()]
xlims=[0.8,1.5]

ylims=[0,10]
#ylims=[0,OGdata.loc[TimeSample,"k_t"].max()]

fig.suptitle(f"k$_{{{'t'}}}$ v.s. α at Time Step {0}")



def animate3(i,alpha, k,net_trade, xlim,ylim):

    plt.clf()
    fig.suptitle(f"k$_{{{'t'}}}$ v.s. α at Time Step {i}")
    plt.xlabel("α")
    plt.ylabel("$k_{t}$")
    pointart=plt.scatter(alpha[i], k[i])
    plt.xlim(xlim)
    plt.ylim(ylim)
    snapshot=str(f"Net Traded:{net_trade[i].mean()}")
    fig.text(0.05, 0.021, snapshot,fontsize=10, bbox=dict(fc="white", ec="white"))





points=plt.scatter(alpha[0], k[0])
    #hue="i_a", 
    #palette=["b", "r","#7f7f7f"], 

plt.xlim(xlims)
plt.ylim(ylims)

anim3 = animation.FuncAnimation(fig, animate3, frames=k.shape[1], interval=2000, blit=False, fargs=(alpha,k,net_trade,xlims,ylims,)) 



plt.show()
'''

ModuleNotFoundError: No module named 'matplotlib'